In [ ]:
from utils.utils import * 
import numpy as np
 
v = np.array([1,2,3])
rho = np.array([5,6,7])

v_hat = SO3_hat(v)
xi = SE3_assemble_xi(v, rho)
xi_hat = SE3_hat(xi)

print(v_hat)
print(xi_hat)
print(xi)
print(xi_hat)

v = SO3_vee(v_hat)
xi = SE3_vee(xi_hat)
print(v)
print(xi)

# We can now 
# phi -> phi_hat and phi_hat -> phi
# xsi -> xsi_hat and xsi_hat -> xsi


In [ ]:
# from utils.utils import * 
# import numpy as np


# phi = np.array([1,2,3])   
# phi_left_jacobian = left_jacobian(phi)
# print(phi_left_jacobian)
# phi_inv_left_jacobian = inv_left_jacobian(phi)
# print(phi_inv_left_jacobian)


In [ ]:
from utils.utils import * 
import numpy as np

# d/dt g(t) = g(t) * xi
# g(t) in SE(3)
# xi in R^6

def g(t, xi_hat, g_0): 
    se3_curve = SE3_dot(g_0, SE3_exp(t * xi_hat))
    se3_curve = np.where(np.abs(se3_curve) < np.finfo(float).eps, 0.0, se3_curve)
    return se3_curve

g_0 = np.eye(4)
xi = np.array([0.1,0,0,1,1,1]) 
xi_hat = SE3_hat(xi)

g_t = g(.1, xi_hat, g_0)
print(is_SE3(g_t))
print(g_t)


In [ ]:
import numpy as np

# Define a rotation matrix for a rotation around the Z-axis by 45 degrees
theta_z = np.radians(45)
R_z = np.array([
    [np.cos(theta_z), -np.sin(theta_z), 0],
    [np.sin(theta_z), np.cos(theta_z), 0],
    [0, 0, 1]
])

# Define a rotation matrix for a rotation around the Y-axis by 90 degrees
theta_y = np.radians(90)
R_y = np.array([
    [np.cos(theta_y), 0, np.sin(theta_y)],
    [0, 1, 0],
    [-np.sin(theta_y), 0, np.cos(theta_y)]
])

# Define translations
t_0 = np.array([1, 0, 0, 1])  # Translation for g_0_0
t_1 = np.array([0, 1, 0, 1])  # Translation for g_0_1

# Construct the SE(3) matrices
g_0_0 = np.eye(4)
g_0_0[:3, :3] = R_z
g_0_0[:, 3] = t_0

g_0_1 = np.eye(4)
g_0_1[:3, :3] = R_y
g_0_1[:, 3] = t_1

g_0_2 = np.eye(4)

g_0_s = {'0' : g_0_0, '1' : g_0_1, '2' : g_0_2}

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Initial vector (tail at origin, head at (1,1,1))
tail_origin = np.array([0, 0, 0, 1])  # Starting on the x-axis
head_origin = np.array([1, 1, 1, 1])  # Making sure it's visible when rotating around the y-axis

w_x, w_y, w_z = 0, 1, 0
v_x, v_y, v_z = 1, 0, 0
xi = np.array([w_x, w_y, w_z, v_x, v_y, v_z])
xi_hat = SE3_hat(xi)

# Transformation matrix g(t), assuming you've calculated it already
g_t = g(1, xi_hat, g_0)

# Apply transformation
tail_transformed = g_t @ tail_origin
head_transformed = g_t @ head_origin

# Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Original vector
ax.quiver(tail_origin[0], tail_origin[1], tail_origin[2], head_origin[0]-tail_origin[0], head_origin[1]-tail_origin[1], head_origin[2]-tail_origin[2], color='r')

for i in np.linspace(0,10,100): 
    g_t = g(i, xi_hat, g_0)
    if not is_SE3(g_t): 
        print("Not SE3")
        print(g_t)
        break
    tail_transformed = g_t @ tail_origin
    head_transformed = g_t @ head_origin
    ax.quiver(tail_transformed[0], tail_transformed[1], tail_transformed[2], head_transformed[0]-tail_transformed[0], head_transformed[1]-tail_transformed[1], head_transformed[2]-tail_transformed[2], color='b')

ax.set_xlim([-3, 3])
ax.set_ylim([-3, 3])
ax.set_zlim([-3, 3])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()


In [ ]:
def create_xi_hat_0(t):
    """
    Create a xi_hat for a given time t

    Input:
        t: float

    Output:
        xi_hat: np.array, shape (4,4)
    """
    a, b = 1, 1
    vx = a * np.cos(10 * t)
    vy = a * np.sin(t / (2 * np.pi))
    vz = b * t
    wx = np.cos(t)
    wy = np.exp(t)
    wz = 1
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def create_xi_hat_1(t):
    """
    Create a xi_hat for a given time t

    Input:
        t: float

    Output:
        xi_hat: np.array, shape (4,4)
    """
    r = 1
    omega = 0.5
    vx = -r * omega * np.sin(omega * t)
    vy = r * omega * np.cos(omega * t)
    vz = 0  # No motion in z direction for a pure circle
    wx = .1  # No rotation around x-axis for a pure circle
    wy = 0  # No rotation around y-axis for a pure circle
    wz = 0  # No rotation around z-axis for a pure circle
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def create_xi_hat_2(t):
    """
    Create a xi_hat for a given time t

    Input:
        t: float

    Output:
        xi_hat: np.array, shape (4,4)
    """
    a, b = 1, 1
    vx = a * np.cos(10 * t)
    vy = 0
    vz = (a+b) * t
    wx = np.sin(t)
    wy = t**2
    wz = 0.1
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def create_xi_hat_3(t):
    """
    Create a xi_hat for a given time t

    Input:
        t: float

    Output:
        xi_hat: np.array, shape (4,4)
    """
    vx = np.log(t + 1)
    vy = 0
    vz = np.exp(t) - 1
    wx = 0
    wy = np.sin(t) * np.cos(t)
    wz = t ** 2 / 10

    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def SE3_create_curve(t, create_xi_hat, g_0):
    c = np.zeros((t.shape[0],4,4))
    for i, t_i in enumerate(t):
        xi_hat = create_xi_hat(t_i)
        c[i] = g(t_i, xi_hat, g_0)
        assert is_SE3(c[i]), f"Not close to SE3, {c[i]}"
    return c

def SE3_move_to_origin(c):
    c_inv = SE3_inv(c[0])
    for i in range(c.shape[0]):
        c[i] = SE3_dot(c_inv, c[i])
    return c

def SE3_right_log_derivative_curve(I, c):
    dc = np.zeros((I.shape[0] - 1, 4, 4))
    for index in range(I.shape[0] - 1):
        dc[index] = SE3_log(SE3_dot(c[index + 1], SE3_inv(c[index]))) / (I[index + 1] - I[index])
    return dc

def SRVT(I, c):
    derivative = SE3_right_log_derivative_curve(I, c)
    norm_derivative = np.linalg.norm(derivative, axis=(1, 2))
    norm_derivative[norm_derivative < np.finfo(float).eps] = 1.0
    norm_derivative = np.sqrt(norm_derivative)
    return np.divide(derivative, norm_derivative[:, None, None])

def SE3_hat_curve(c):
    if len(c.shape) == 3:
        return np.array([SE3_hat_curve(c_i) for c_i in c])
    return SE3_vee(c)

def SE3_prepare_curve(I, c):
    c = SE3_move_to_origin(c)
    c = SRVT(I, c)
    q = SE3_hat_curve(c)
    return q

functions = {
    'identity': lambda t: t,
    "sin": lambda t: np.sin(t) / np.sin(1),
    "square": lambda t: t**2,
    "log": lambda t: np.log(t + 1) / np.log(2),
    "sqrt": lambda t: np.sqrt(t),
    "exp": lambda t: (np.exp(t) - 1) / (np.exp(1) - 1),
}

f = functions["sin"]

g_0 = np.eye(4)
timesteps = 4
t_eq = np.linspace(0,1,timesteps)


t0 = f(t_eq)
c0 = SE3_create_curve(t0, create_xi_hat_0, g_0)
q0 = SE3_prepare_curve(t_eq, c0)

t1 = t_eq.copy()
c1 = SE3_create_curve(t1, create_xi_hat_0, g_0)
q1 = SE3_prepare_curve(t_eq, c1)

In [ ]:
import sys
sys.path.append('..')

from SO3.utils.reparameterization_utils import find_optimal_diffeomorphism, L2_metric

print(f"Original L2 {L2_metric(q0, q1, t_eq, t_eq)}")

for depth in range(2, 10):
    t1_new = find_optimal_diffeomorphism(q0, q1, t_eq, t_eq, depth)
    c1_new = SE3_reparameterize(t1_new, t_eq, c1)
    q1_new = SE3_prepare_curve(t_eq, c1_new)

    print(f"Depth {depth}; L2 {L2_metric(q0, q1_new, t_eq, t_eq)}")

plt.plot(t_eq, t_eq, label=r"$I$", linestyle="--")
plt.plot(t_eq, t0, label=r"$\varphi(I)$")
plt.plot(t_eq, t1_new, label=r"$\psi(I)$", color = "red")
# plt.plot(t1_new, t_eq, label=r"$\psi(I)$ mirrored", color="blue")

plt.legend()
plt.show()


In [ ]:
depth = 8
t1_new = find_optimal_diffeomorphism(q0, q1, t_eq, t_eq, depth)
c1_new = SE3_reparameterize(t1_new, t_eq, c1)
q1_new = SE3_prepare_curve(t_eq, c1_new)

plt.plot(t_eq, t_eq, label=r"$I$", linestyle="--")
plt.plot(t_eq, t0, label=r"$\varphi(I)$")
plt.plot(t_eq, t1_new, label=r"$\psi(I)$", color = "red")
# plt.plot(t1_new, t_eq, label=r"$\psi(I)$ mirrored", color="blue")

plt.legend()
plt.show()

In [ ]:
def create_and_prepare_SE3_curve(g_0, t_eq, f, create_xi_hat): 
    t = f(t_eq)
    c = SE3_create_curve(t, create_xi_hat, g_0)
    q = SE3_prepare_curve(t_eq, c)
    return t, c, q

# I need two different g_0s
# I need two different xi_hats
# I will start with two different g_0s
g_0_0 = g_0_s['0']
g_0_1 = g_0_s['1']
num_timesteps = 50
t_eq = np.linspace(0,1,num_timesteps)
t0, c0, q0 = create_and_prepare_SE3_curve(g_0_0, t_eq, functions['sin'], create_xi_hat_0)
t1, c1, q1 = create_and_prepare_SE3_curve(g_0_1, t_eq, functions['identity'], create_xi_hat_0)
t2, c2, q2 = create_and_prepare_SE3_curve(g_0_0, t_eq, functions['identity'], create_xi_hat_0)

# We will now reparemeterize curve 1 and 2 to curve 0

print("Original L2")
print(f" c0 v c1 {L2_metric(q0, q1, t_eq, t_eq)}")
print(f" c0 v c2 {L2_metric(q0, q2, t_eq, t_eq)}")

for depth in range(2, 9):
    t1_new = find_optimal_diffeomorphism(q0, q1, t_eq, t_eq, depth)
    c1_new = SE3_reparameterize(t1_new, t_eq, c1)
    q1_new = SE3_prepare_curve(t_eq, c1_new)

    t2_new = find_optimal_diffeomorphism(q0, q2, t_eq, t_eq, depth)
    c2_new = SE3_reparameterize(t2_new, t_eq, c2)
    q2_new = SE3_prepare_curve(t_eq, c2_new)

    print(f"Depth {depth}; L2")
    print(f" c0 v c1 {L2_metric(q0, q1_new, t_eq, t_eq)}")
    print(f" c0 v c2 {L2_metric(q0, q2_new, t_eq, t_eq)}")

plt.plot(t_eq, t_eq, label=r"$I$", linestyle="--")
plt.plot(t_eq, t0, label=r"$\varphi(I)$")
plt.plot(t_eq, t1_new, label=r"$\psi(I)$", color = "red")
plt.plot(t_eq, t2_new, label=r"$\rho(I)$", color = "blue")
plt.legend()
plt.show()

In [ ]:
def create_and_prepare_SE3_curve(g_0, t_eq, f, create_xi_hat): 
    t = f(t_eq)
    c = SE3_create_curve(t, create_xi_hat, g_0)
    q = SE3_prepare_curve(t_eq, c)
    return t, c, q

# I need two different g_0s
# I need two different xi_hats
# I will start with two different g_0s
g_0_0 = g_0_s['0']
g_0_1 = g_0_s['1']
num_timesteps = 50
t_eq = np.linspace(0,1,num_timesteps)
t0, c0, q0 = create_and_prepare_SE3_curve(g_0_0, t_eq, functions['sin'], create_xi_hat_0)
t1, c1, q1 = create_and_prepare_SE3_curve(g_0_0, t_eq, functions['identity'], create_xi_hat_1)
t2, c2, q2 = create_and_prepare_SE3_curve(g_0_0, t_eq, functions['identity'], create_xi_hat_0)

# We will now reparemeterize curve 1 and 2 to curve 0

print("Original L2")
print(f" c0 v c1 {L2_metric(q0, q1, t_eq, t_eq)}")
print(f" c0 v c2 {L2_metric(q0, q2, t_eq, t_eq)}")

for depth in range(7, 9):
    t1_new = find_optimal_diffeomorphism(q0, q1, t_eq, t_eq, depth)
    c1_new = SE3_reparameterize(t1_new, t_eq, c1)
    q1_new = SE3_prepare_curve(t_eq, c1_new)

    t2_new = find_optimal_diffeomorphism(q0, q2, t_eq, t_eq, depth)
    c2_new = SE3_reparameterize(t2_new, t_eq, c2)
    q2_new = SE3_prepare_curve(t_eq, c2_new)

    print(f"Depth {depth}; L2")
    print(f" c0 v c1 {L2_metric(q0, q1_new, t_eq, t_eq)}")
    print(f" c0 v c2 {L2_metric(q0, q2_new, t_eq, t_eq)}")

plt.plot(t_eq, t_eq, label=r"$I$", linestyle="--")
plt.plot(t_eq, t0, label=r"$\varphi(I)$")
plt.plot(t_eq, t1_new, label=r"$\psi(I)$", color = "red")
plt.plot(t_eq, t2_new, label=r"$\rho(I)$", color = "blue")
plt.legend()
plt.show()

In [ ]:
plt.plot(t_eq, t_eq, label=r"$I$", marker='.', linewidth=1)
plt.plot(t_eq, t0, label=r"$\varphi(I)$", marker='.', linewidth=1)
plt.plot(t_eq, t1_new, label=r"$\psi(I)$", color = "red", marker='.', linewidth=1)
plt.plot(t_eq, t2_new, label=r"$\rho(I)$", color = "blue", marker='.', linewidth=1)
plt.legend()
plt.show()

In [ ]:
def create_xi_hat_0(t):
    """
    A variation with linearly increasing angular velocity and sinusoidal linear velocity.
    """
    vx = np.sin(t * 11)
    vy = np.cos(t)
    vz = 0
    wx = t
    wy = 0
    wz = 0
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def create_xi_hat_1(t):
    """
    A variation with linearly increasing angular velocity and sinusoidal linear velocity.
    """
    vx = np.sin(t * 10)
    vy = np.cos(t)
    vz = 0
    wx = t
    wy = 0
    wz = 0
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def create_xi_hat_2(t):
    """
    A variation with linearly increasing angular velocity and sinusoidal linear velocity.
    """
    vx = np.sin(t * 9)
    vy = np.cos(t)
    vz = 0
    wx = t
    wy = 0
    wz = 0
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))

def create_xi_hat_3(t):
    """
    A variation with linearly increasing angular velocity and sinusoidal linear velocity.
    """
    vx = np.sin(t * 8)
    vy = np.cos(t)
    vz = 0
    wx = t
    wy = 0
    wz = 0
    return SE3_hat(np.array([wx, wy, wz, vx, vy, vz]))



In [ ]:
# I need 3 different xi_hats and 3 different g_0s resulting in 9 different shapes and 3 different reparemeterizations s.t. we get 27 curves
num_timesteps = 50
t_eq = np.linspace(0,1,num_timesteps)
xi_hats = [create_xi_hat_0, create_xi_hat_1, create_xi_hat_2, create_xi_hat_3]
params = ['sin', 'identity', 'exp']
curves = {}
for i, xi_hat_func in enumerate(xi_hats): 

    for param in params:
        t, c, q = create_and_prepare_SE3_curve(g_0, t_eq, functions[param], xi_hat_func)
        curves[f"{i}_{param}"] = {'t': t, 'c': c, 'q': q}


In [ ]:
# Will now reparameterize all curves to the first curves
depth = 6
distances = np.zeros((len(curves), len(curves)))
distances_rep = np.zeros((len(curves), len(curves)))
numbers = {i: key.split('_')[0] for i, key in enumerate(curves.keys())}
parameterizations = {i: key.split('_')[1] for i, key in enumerate(curves.keys())}
ids = {i: {'curve' : numbers[i], 'param' : parameterizations[i]} for i in range(len(curves))}
for i, (key0, value0) in enumerate(curves.items()): 
    for j, (key1, value1) in enumerate(curves.items()):
        if key0 == key1: 
            distances[i, j] = 0
            distances_rep[i, j] = 0
        if i >= j: 
            continue
        
        t0, c0, q0 = value0['t'], value0['c'], value0['q']
        t1, c1, q1 = value1['t'], value1['c'], value1['q']

        distances[i, j] = L2_metric(q0, q1, t_eq, t_eq)
        distances[j, i] = distances[i, j]

        # We will fit q1 to q0, and store the L2 distance
        t1_new = find_optimal_diffeomorphism(q0, q1, t_eq, t_eq, depth)
        c1_new = SE3_reparameterize(t1_new, t_eq, c1)
        q1_new = SE3_prepare_curve(t_eq, c1_new)
        distances_rep[i, j] = L2_metric(q0, q1_new, t_eq, t_eq)
        distances_rep[j, i] = distances_rep[i, j]

        print(f"{key0} v {key1} done")


In [ ]:
import sys 
sys.path.append('../..')

from SO3.utils.post_processing_utils import cMDS

coordinates = cMDS(distances)
coordinates_rep = cMDS(distances_rep)

import matplotlib.patches as mpatches

# Existing code
colors = [item['curve'] for item in ids.values()]
cmap = plt.cm.get_cmap('viridis', len(set(colors)))
color_nums = [int(color) for color in colors]

# Plot the original distances in 2D
plt.scatter(coordinates[:, 0], coordinates[:, 1], c=color_nums, cmap=cmap)
# Create legend
unique_colors = set(colors)
patches = [mpatches.Patch(color=cmap(int(color)), label=color) for color in unique_colors]
plt.legend(handles=patches)
plt.show()

# Plot the reparameterized distances in 2D
plt.scatter(coordinates_rep[:, 0], coordinates_rep[:, 1], c=color_nums, cmap=cmap)
# Create legend
unique_colors = set(colors)
patches = [mpatches.Patch(color=cmap(int(color)), label=color) for color in unique_colors]
plt.legend(handles=patches)
plt.show()

In [ ]:
print(distances)

In [ ]:
print(distances_rep)

In [ ]:
import sys 
sys.path.append('..')

from utils.visualize import SE3_transform_curve, SE3_visualize_curve

points, orientations = SE3_transform_curve(c0)
scale = 1

SE3_visualize_curve(points, orientations, scale)

In [ ]:
import sys 
sys.path.append('..')

from utils.visualize import SE3_transform_curve, SE3_visualize_curve

points, orientations = SE3_transform_curve(c1)
scale = 1

SE3_visualize_curve(points, orientations, scale)

In [ ]:
c_2 = curves["1_sin"]['c']
c_3 = curves["1_identity"]['c']
c_4 = curves["1_exp"]['c']

points, orientations = SE3_transform_curve(c_2)
scale = 1

SE3_visualize_curve(points, orientations, scale)

points, orientations = SE3_transform_curve(c_3)
scale = 1

SE3_visualize_curve(points, orientations, scale)

points, orientations = SE3_transform_curve(c_4)
scale = 1

SE3_visualize_curve(points, orientations, scale)

In [ ]:
t0, c0, q0 = curves["1_sin"]['t'], curves["1_sin"]['c'], curves["1_sin"]['q']
t1, c1, q1 = curves["1_identity"]['t'], curves["1_identity"]['c'], curves["1_identity"]['q']

print(q0)

print("Original L2")
print(f" c0 v c1 {L2_metric(q0, q1, t_eq, t_eq)}")
print(f" c0 v c2 {L2_metric(q0, q2, t_eq, t_eq)}")

for depth in range(7, 8):
    t1_new = find_optimal_diffeomorphism(q0, q1, t_eq, t_eq, depth)
    c1_new = SE3_reparameterize(t1_new, t_eq, c1)
    q1_new = SE3_prepare_curve(t_eq, c1_new)



    print(f"Depth {depth}; L2")
    print(f" c0 v c1 {L2_metric(q0, q1_new, t_eq, t_eq)}")

plt.plot(t_eq, t_eq, label=r"$I$", linestyle="--")
plt.plot(t_eq, t0, label=r"$\varphi(I)$")
plt.plot(t_eq, t1_new, label=r"$\psi(I)$", color = "red")
plt.legend()
plt.show()